In [15]:
import requests
from io import StringIO
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

In [18]:
# URL의 내용 가져오기
url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/d72b9558a11523adbe13300b41321ecd93d331d3/document.txt"
response = requests.get(url)
text = response.text

In [20]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

# 텍스트를 청크로 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# 분할된 텍스트에서 문서 생성
documents = [Document(page_content=t) for t in texts]

Created a chunk of size 1444, which is longer than the specified 1000
Created a chunk of size 1251, which is longer than the specified 1000
Created a chunk of size 1012, which is longer than the specified 1000
Created a chunk of size 2313, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 1673, which is longer than the specified 1000
Created a chunk of size 1137, which is longer than the specified 1000
Created a chunk of size 1559, which is longer than the specified 1000
Created a chunk of size 1200, which is longer than the specified 1000
Created a chunk of size 1042, which is longer than the specified 1000
Created a chunk of size 1345, which is longer than the specified 1000
Created a chunk of size 1339, which is longer than the specified 1000
Created a chunk of size 1288, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of s

In [22]:
from langchain.docstore.document import Document

# 임베딩 생성 및 Chroma에 문서 저장
embeddings = OpenAIEmbeddings(openai_api_key="")

# 문자열을 Document 객체로 변환
documents = [Document(page_content=t) for t in texts]

docsearch = Chroma.from_documents(documents, embeddings)

In [24]:
# ConversationBufferMemory를 사용하여 체인 생성
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(openai_api_key=""),
    docsearch.as_retriever(),
    memory=memory
)

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

# ConversationBufferMemory를 사용하여 체인 생성
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=""),
    docsearch.as_retriever(),
    memory=memory
)

# 체인에 질문하고 답변 출력
questions = [
    "Aaronson은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?", 
    "Julia는 누구인가요?"
]

chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"질문: {question}")
    print(f"답변: {result['answer']}\n")
    
#끝!!

질문: Aaronson은 유죄인가요?
답변: 죄인인지 여부를 알 수 없습니다.

질문: 그가 테이블에 어떤 메시지를 썼나요?
답변: 그 문맥에서는 그가 테이블에 어떤 메시지를 쓴 것에 대한 언급이 없습니다.

질문: Julia는 누구인가요?
답변: Julia는 위스키스미스의 소설 '1984'에서 중요한 캐릭터로 나오는 여성 캐릭터입니다. 위 스크립트에서는 주인공인 윈스턴이 Julia를 사랑하는 모습이 묘사되어 있습니다. Julia는 윈스턴과 함께 반인권 정부에 저항하고 사랑을 나누는데 중요한 역할을 합니다.



In [ ]:
#Rmx